<a href="https://colab.research.google.com/github/cundeyu154/PL-Repo/blob/main/%E4%BD%9C%E6%A5%AD%E4%BA%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gspread pandas gradio google-genai
import gspread
import pandas as pd
# 🚨 關鍵變動：新增 userdata 才能從 Colab Secrets 獲取 API Key
from google.colab import auth, userdata
import datetime
import gradio as gr
import os
import numpy as np
# 🚨 關鍵變動：新增 Gemini API 匯入
from google import genai
from google.genai.errors import APIError

print("✅ 函式庫安裝與匯入完成！")

spreadsheet_name = "HW1日常支出數算與分攤"
worksheet_name = "Sheet1"
gc = None

def authenticate_and_get_client():
    """處理 Google 授權並取得 gspread 客戶端。"""
    global gc
    if gc is None:
        try:
            print("\n請點擊連結完成 Google 授權：")
            auth.authenticate_user()
            print("Google 授權完成！")
            from google.auth import default
            creds, _ = default()
            gc = gspread.authorize(creds)
            print("gspread 授權成功。")
            return gc
        except Exception as e:
            print(f"❌ 授權失敗: {e}")
            return None
    else:
        return gc

gc = authenticate_and_get_client()

# -------------------- 🚨 關鍵變動：設置 API KEY 🚨 --------------------
# 警告：請務必先在 Google Colab 左側邊欄（鑰匙圖標）中，
# 創建一個名為 GEMINI_API_KEY 的 Secret，並貼入您的 Key。
try:
    os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')
    print("✅ 已從 Colab Secrets 載入 GEMINI_API_KEY。")
except Exception:
    print("⚠️ 找不到 Colab Secret (GEMINI_API_KEY)。請確認您已設定 Secret。AI 分析功能將無法使用。")
# --------------------------------------------------------------------


def run_analysis(gc):
    """
    從 Google Sheets 讀取資料，執行計算，透過 Gemini API 產生分析文字，
    並將所有結果 (包含分析文字) 寫入 Summary 分頁。
    """
    if gc is None:
        return pd.DataFrame(), pd.DataFrame(), "❌ 錯誤：gspread 尚未授權。請先完成授權步驟。"

    try:
        sh = gc.open(spreadsheet_name)
        worksheet = sh.worksheet(worksheet_name)
        try:
            summary_ws = sh.worksheet("Summary")
        except gspread.WorksheetNotFound:
            summary_ws = sh.add_worksheet(title="Summary", rows="100", cols="20")

        print(f"試算表 '{spreadsheet_name}' 連接成功，開始讀取數據...")
        data = worksheet.get_all_records()
        df = pd.DataFrame(data)

        if df.empty:
            return pd.DataFrame(), pd.DataFrame(), "⚠️ 數據分頁為空，無法進行分析。"

        df['金額'] = pd.to_numeric(df['金額'], errors='coerce')
        df.dropna(subset=['金額'], inplace=True)

        if df.empty:
            return pd.DataFrame(), pd.DataFrame(), "⚠️ 數據分頁中 '金額' 欄位無有效數字，無法進行分析。"

        # --- 計算部分 ---
        total_amount = df['金額'].sum()
        total_records = len(df)
        category_subtotals = df.groupby('分類')['金額'].sum().reset_index()
        category_subtotals.columns = ['分類', '金額小計']

        all_payers = df['付款人'].unique()
        num_payers = len(all_payers)
        average_share = total_amount / num_payers if num_payers > 0 else 0

        payer_payments = df.groupby('付款人')['金額'].sum().reindex(all_payers, fill_value=0).reset_index()
        payer_payments.columns = ['付款人', '已付總額']
        payer_payments['應分攤金額'] = average_share
        payer_payments['淨額'] = payer_payments['已付總額'] - payer_payments['應分攤金額']

        # 準備 Gradio 輸出 (金額格式化)
        payer_payments_gradio = payer_payments.copy()
        payer_payments_gradio[['已付總額', '應分攤金額', '淨額']] = payer_payments_gradio[['已付總額', '應分攤金額', '淨額']].applymap('{:,.2f}'.format)

        # --- 🚨 關鍵變動：使用 Gemini API 進行 AI 分析 🚨 ---
        try:
            # 1. 初始化 Gemini 客戶端
            client = genai.Client()

            # 2. 準備提示詞 (Prompt)
            prompt_data = f"""
            請你扮演一位財務分析師，根據以下日常支出數據，撰寫一份簡潔的分析報告，提供實用建議。

            --- 原始數據摘要 ---
            總消費額: NT${total_amount:,.2f}
            總筆數: {total_records} 筆
            每人平均應分攤: NT${average_share:,.2f}

            --- 分類支出小計 ---
            {category_subtotals.to_string(index=False)}

            --- AA 分攤結果 (淨額: 正數為應收, 負數為應付) ---
            {payer_payments.to_string(index=False)}

            --- 報告要求 ---
            1. 報告標題請用「💡 日常支出 AI 深度分析 💡」。
            2. 概述總支出與最高支出分類，並提出 1-2 條節流建議。
            3. 根據「淨額」欄位，明確指出誰應付最多和誰應收最多，並提供結算建議。
            4. 使用繁體中文，且回覆內容請不要超過 200 字。
            """

            # 3. 呼叫 Gemini 模型
            response = client.models.generate_content(
                model='gemini-2.5-flash',
                contents=prompt_data
            )

            analysis_text = response.text.strip()
            print("✅ 成功呼叫 Gemini API 並生成分析報告。")

        except APIError as e:
            analysis_text = f"❌ Gemini API 呼叫失敗。請檢查您的 API Key (GEMINI_API_KEY) 是否正確設置。錯誤資訊: {e}"
            print(analysis_text)
        except Exception as e:
            # 這是捕捉其他錯誤，例如沒有設置 API Key 環境變數
            analysis_text = f"❌ AI 服務發生未知錯誤。請確認您已設置 API Key。錯誤資訊: {e}"
            print(analysis_text)
        # ------------------------------------------------------------------

        # --- 寫入 Google Sheets Summary 分頁 (這部分與你的原邏輯一致，用於存儲) ---
        summary_ws.clear()

        # 1. 基礎資訊
        summary_ws.update_cell(1, 1, "總結報告")
        summary_ws.update_cell(2, 1, "總消費額")
        summary_ws.update_cell(2, 2, float(total_amount))
        summary_ws.update_cell(3, 1, "總筆數")
        summary_ws.update_cell(3, 2, total_records)
        summary_ws.update_cell(4, 1, "每人平均應分攤")
        summary_ws.update_cell(4, 2, float(average_share))

        # 2. 分類小計 (A6 開始)
        summary_ws.update_cell(6, 1, "分類小計")
        subtotals_list = [category_subtotals.columns.tolist()] + category_subtotals.values.tolist()
        summary_ws.update(range_name=f'A7:B{7 + len(subtotals_list) - 1}', values=subtotals_list)

        # 3. AA 分攤結果 (D6 開始)
        summary_ws.update_cell(6, 4, "AA 分攤結果")
        aa_list = [payer_payments.columns.tolist()] + payer_payments.values.tolist()
        summary_ws.update(range_name=f'D7:G{7 + len(aa_list) - 1}', values=aa_list)

        # 4. 將 AI 分析文字寫入 Summary 分頁 (A15 開始)
        ai_title = "💡 AI 分析報告全文 (Gradio Output)"
        # 移除 Markdown 符號，讓內容更適合 Sheets 儲存格
        clean_text = analysis_text.replace('**', '').replace('\n---', '\n\n---').replace('\n👉', '\n\n👉')

        summary_ws.update_cell(15, 1, ai_title)
        summary_ws.update_cell(16, 1, clean_text)
        print("✅ AI 分析報告已寫入 Summary 分頁的 A16 儲存格！")

        print("✅ 所有結果已成功寫入 Google Sheets 的 Summary 分頁！")

        return category_subtotals, payer_payments_gradio[['付款人', '已付總額', '應分攤金額', '淨額']], analysis_text

    except gspread.WorksheetNotFound:
        return pd.DataFrame(), pd.DataFrame(), f"❌ 錯誤：找不到資料分頁: {worksheet_name}。請檢查分頁名稱。"
    except gspread.SpreadsheetNotFound:
        return pd.DataFrame(), pd.DataFrame(), f"❌ 錯誤：找不到試算表: {spreadsheet_name}。請檢查檔案名稱。"
    except Exception as e:
        return pd.DataFrame(), pd.DataFrame(), f"❌ 發生未知錯誤: {e}"

def gradio_wrapper():
    """確保每次都使用全域的 gc 客戶端來呼叫分析函數。"""
    global gc
    return run_analysis(gc)

with gr.Blocks(title="日常支出數算與分攤報告") as demo:
    gr.Markdown("# 💸 Google Sheets 日常支出數算與分攤報告 📊")

    # 設置為 Markdown 輸出，可以解析 analysis_text 中的粗體等格式
    ai_analysis_output = gr.Markdown(label="AI 分析結果", value="點擊下方按鈕開始分析...")
    run_button = gr.Button("點擊我更新並執行分析")

    gr.Markdown("---")
    gr.Markdown("## 📋 計算詳細結果")

    with gr.Row():
        category_df_output = gr.DataFrame(
            headers=["分類", "金額小計"],
            label="分類支出小計"
        )
        aa_df_output = gr.DataFrame(
            headers=["付款人", "已付總額", "應分攤金額", "淨額"],
            label="AA 分攤淨額 (+號為應收, -號為應付)"
        )

    # 定義互動邏輯 (按鈕點擊觸發分析)
    run_button.click(
        fn=gradio_wrapper,
        inputs=None,
        outputs=[category_df_output, aa_df_output, ai_analysis_output]
    )

    # 在頁面加載時自動執行一次分析
    demo.load(
        fn=gradio_wrapper,
        inputs=None,
        outputs=[category_df_output, aa_df_output, ai_analysis_output]
    )

# 啟動 Gradio 介面
print("\n--- 啟動 Gradio 介面 ---")
demo.launch(debug=True, share=True)

✅ 函式庫安裝與匯入完成！

請點擊連結完成 Google 授權：
Google 授權完成！
gspread 授權成功。
✅ 已從 Colab Secrets 載入 GEMINI_API_KEY。

--- 啟動 Gradio 介面 ---
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f0297529c8087eee89.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


試算表 'HW1日常支出數算與分攤' 連接成功，開始讀取數據...


/tmp/ipython-input-92713751.py:98: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  payer_payments_gradio[['已付總額', '應分攤金額', '淨額']] = payer_payments_gradio[['已付總額', '應分攤金額', '淨額']].applymap('{:,.2f}'.format)


✅ 成功呼叫 Gemini API 並生成分析報告。
✅ AI 分析報告已寫入 Summary 分頁的 A16 儲存格！
✅ 所有結果已成功寫入 Google Sheets 的 Summary 分頁！
試算表 'HW1日常支出數算與分攤' 連接成功，開始讀取數據...


/tmp/ipython-input-92713751.py:98: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  payer_payments_gradio[['已付總額', '應分攤金額', '淨額']] = payer_payments_gradio[['已付總額', '應分攤金額', '淨額']].applymap('{:,.2f}'.format)


✅ 成功呼叫 Gemini API 並生成分析報告。
✅ AI 分析報告已寫入 Summary 分頁的 A16 儲存格！
✅ 所有結果已成功寫入 Google Sheets 的 Summary 分頁！
試算表 'HW1日常支出數算與分攤' 連接成功，開始讀取數據...


/tmp/ipython-input-92713751.py:98: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  payer_payments_gradio[['已付總額', '應分攤金額', '淨額']] = payer_payments_gradio[['已付總額', '應分攤金額', '淨額']].applymap('{:,.2f}'.format)


✅ 成功呼叫 Gemini API 並生成分析報告。
✅ AI 分析報告已寫入 Summary 分頁的 A16 儲存格！
✅ 所有結果已成功寫入 Google Sheets 的 Summary 分頁！
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://f0297529c8087eee89.gradio.live
